- - -
Read from file containing previously generated brute force color vector with fixed $q_2$ few initial color vectors for $q_3$ values of interest. 

In [1]:
import pickle
import numpy as np

# q2 = 4.0
q2 = 1.33
brute_force_q = pickle.load(open('results/brute_force_q2_' + str(q2) + '.pickle', 'rb'))
q3s, q0s = brute_force_q["q3s"], brute_force_q["q0s"]

In [2]:
# test
# q3s_sim = [0]
if q2==4.0:
    q3s_sim = [0, 1, 2, 3, 4]
else:
    q3s_sim = [0, 0.2, 0.4, 0.6, 0.8]

def find_nearest(array, value):
    array = np.asarray(array)
    idx = (np.abs(array - value)).argmin()
    return idx

q0s_sim = np.zeros((len(q3s_sim), 8))
for (q3index, q3value) in enumerate(q3s_sim):
    q0s_sim[q3index] = q0s[find_nearest(q3s, q3value)]

In [3]:
from scipy.stats import unitary_group

# gell-mann matrices

gm = [
    [[0, 1, 0], [1, 0, 0], [0, 0, 0]],
    [[0, -1j, 0], [1j, 0, 0], [0, 0, 0]],
    [[1, 0, 0], [0, -1, 0], [0, 0, 0]],
    [[0, 0, 1], [0, 0, 0], [1, 0, 0]],
    [[0, 0, -1j], [0, 0, 0], [1j, 0, 0]],
    [[0, 0, 0], [0, 0, 1], [0, 1, 0]],
    [[0, 0, 0], [0, 0, -1j], [0, 1j, 0]],
    [[1 / np.sqrt(3), 0, 0], [0, 1 / np.sqrt(3), 0], [0, 0, -2 / np.sqrt(3)]]
]

T = np.array(gm) / 2.0

def init_charge(q0):
    """
        Step 1: specific random color vector
    """
    Q0 = np.einsum('ijk,i', T, q0)

    """
        Step 2: create a random SU(3) matrix to rotate Q.
    """
    
    V = unitary_group.rvs(3)
    detV = np.linalg.det(V)
    U = V / detV ** (1 / 3)
    Ud = np.conj(U).T

    Q = np.einsum('ab,bc,cd', U, Q0, Ud)

    """
        Step 3: Project onto color components
    """

    q = 2 * np.einsum('ijk,kj', T, Q)
    return np.real(q)

- - -

In [4]:
import numpy as np

# hbar * c [GeV * fm]
hbarc = 0.197326 

# Simulation box 
L = 10      
N = 512  
# N = 1024  
tau_sim = 1.5
DTS = 8     

# Glasma fields
su_group = 'su3'
# q2 = 4

Qs = 2        
ns = 50    
factor = 0.8        
g2mu = Qs / factor     
g = np.pi * np.sqrt(1 / np.log(Qs / 0.2))          		
mu = g2mu / g**2          	
ir = 0.1 * g**2 * mu  
 
uv = 10.0           

# Wong
quark = 'charm'    
mass = 1.275       
tau_form = 1/(2*mass)*hbarc

tau_sim += tau_form

# tau_form = 0
initialization = 'toy'         
pT = 2    
ntp = 10**5  
nevents = 1    
representation = 'fundamental'      
boundary = 'periodic'       

# Results folder
folder = 'corr_' + initialization + '_pT_' + str(pT) + '_' + quark + '_q2_' + str(q2)      

# Store relevant parameters in a dictionary
p = {
    'QUARK': quark,
    'MASS': mass,           
    'TFORM': tau_form,   
    'NTP':  ntp,    
    'PT': pT,          
    'FOLDER': folder,         
    }   

In [5]:
import os
os.environ["MY_NUMBA_TARGET"] = "cuda"
os.environ["PRECISION"] = "double"
os.environ["GAUGE_GROUP"] = su_group

# Import relevant modules
import sys
sys.path.append('..')

# Glasma modules
import curraun.core as core
import curraun.mv as mv
import curraun.initial as initial
initial.DEBUG = False

import curraun.su as su
from curraun.numba_target import use_cuda
if use_cuda:
    from numba import cuda

# Wong modules
from curraun import wong
wong.BOUNDARY = boundary
wong.CUB_MOM = False
from curraun.wong import init_mom_toy, init_pos

from curraun.particles_correlators import Angles

Using CUDA
Using SU(3)
Using double precision
Using double precision


In [6]:
import pickle
from tqdm import tqdm

# Simulation routine
def simulate(p, ev):
    # q2, q3, q0 = p["q2"], p["q3"], p["q0"]
    q0 = p["q0"]

    output = {}
    output["parameters"] = p.copy()

    # Derived parameters
    a = L / N
    E0 = N / L * hbarc
    DT = 1.0 / DTS
    formt = int(tau_form / a * DTS)
    maxt = int(tau_sim / a * DTS)

    # Initialize Glasma fields
    s = core.Simulation(N, DT, g)
    va = mv.wilson(s, mu=mu / E0, m=ir / E0, uv=uv / E0, num_sheets=ns)
    vb = mv.wilson(s, mu=mu / E0, m=ir / E0, uv=uv / E0, num_sheets=ns)
    initial.init(s, va, vb)

    # Initialize the Wong solver
    wong_solver_q = wong.WongSolver(s, ntp)
    wong_solver_aq = wong.WongSolver(s, ntp)
    x0s_q, p0s_q, q0s_q = np.zeros((ntp, 3)), np.zeros((ntp, 5)), np.zeros((ntp, su.ALGEBRA_ELEMENTS))
    x0s_aq, p0s_aq, q0s_aq = np.zeros((ntp, 3)), np.zeros((ntp, 5)), np.zeros((ntp, su.ALGEBRA_ELEMENTS))
    masses = mass / E0 * np.ones(ntp)

    for i in range(ntp):
        if initialization=='toy':
            p0_q = init_mom_toy('pT', pT / E0)            

        p0_aq = [-p for p in p0_q]
        p0s_q[i, :], p0s_aq[i, :] = np.array(p0_q, dtype=object), np.array(p0_aq, dtype=object)

        x0_q, q0_q = init_pos(s.n), init_charge(q0)
        x0s_q[i, :], q0s_q[i, :] = x0_q, q0_q
        x0_aq, q0_aq = x0_q, init_charge(q0)
        x0s_aq[i, :], q0s_aq[i, :] = x0_aq, q0_aq

    wong_solver_q.initialize(x0s_q, p0s_q, q0s_q, masses)
    wong_solver_aq.initialize(x0s_aq, p0s_aq, q0s_aq, masses)

    qqbar_angle = Angles(wong_solver_q, wong_solver_aq, ntp)

    deta, dphi = np.zeros((maxt-formt, ntp)), np.zeros((maxt-formt, ntp))
    pTs = np.zeros((maxt-formt, ntp, 2))

    with tqdm(total=maxt) as pbar:
        for t in range(maxt):
            # Evolve Glasma fields
            core.evolve_leapfrog(s)

            # Solve Wong's equations
            if t>=formt:  
                qqbar_angle.compute()
                deta[t-formt] = qqbar_angle.deta.copy()
                dphi[t-formt] = qqbar_angle.dphi.copy()
                pTs[t-formt] = qqbar_angle.pT.copy() * E0
                
                wong_solver_q.evolve()
                wong_solver_aq.evolve()

            pbar.set_description("Event " + str(ev+1))
            pbar.update(1)

    tau = np.linspace(0, tau_sim-p['TFORM'], maxt-formt)
    output['tau'] = tau
    output['deta'], output['dphi'], output['pTs'] = deta, dphi, pTs

    return output

In [7]:
for ev in range(nevents):
    output = {}
    output["q2"] = q2
    output["q3s"] = q3s_sim
    for (q3index, q3value) in enumerate(q3s_sim):
        p["q0"] = q0s_sim[q3index, :]
        print('q3=', q3value)
    
        output[q3value] = simulate(p, ev)


    current_path = os.getcwd() 
    results_folder = 'results'
    if not os.path.isdir(results_folder):
        os.makedirs(results_folder)
    results_path = current_path + '/' + results_folder
    os.chdir(results_path)

    wong_folder = p['FOLDER']
    if not os.path.isdir(wong_folder):
        os.makedirs(wong_folder)
    wong_path = results_path + '/' + wong_folder + '/'
    os.chdir(wong_path)

    filename = 'event_' + str(ev+1) + '.pickle'
    with open(filename, 'wb') as handle:
        pickle.dump(output, handle)  

q3= 0


Event 1: 100%|██████████| 646/646 [04:21<00:00,  2.47it/s]


q3= 0.2


Event 1: 100%|██████████| 646/646 [04:41<00:00,  2.29it/s]


q3= 0.4


Event 1: 100%|██████████| 646/646 [04:48<00:00,  2.24it/s]


q3= 0.6


Event 1: 100%|██████████| 646/646 [04:51<00:00,  2.22it/s]


q3= 0.8


Event 1: 100%|██████████| 646/646 [04:50<00:00,  2.22it/s]


- - -
### Dependence of correlation on initial $p_T$ or $Q_s$

In [1]:
import pickle
import numpy as np

# ATM run with q2=4
q2 = 4.0
# q2 = 1.33
brute_force_q = pickle.load(open('results/brute_force_q2_' + str(q2) + '.pickle', 'rb'))
q3s, q0s = brute_force_q["q3s"], brute_force_q["q0s"]

# ATM runt with null q3, perhaps it is not worth it to choose it as random or equal to C3(F)/D(F)
# In the end, results weakly depend on the value of q3
q3 = 0

def find_nearest(array, value):
    array = np.asarray(array)
    idx = (np.abs(array - value)).argmin()
    return idx

q0 = q0s[find_nearest(q3s, q3)]

In [2]:
from scipy.stats import unitary_group

# gell-mann matrices

gm = [
    [[0, 1, 0], [1, 0, 0], [0, 0, 0]],
    [[0, -1j, 0], [1j, 0, 0], [0, 0, 0]],
    [[1, 0, 0], [0, -1, 0], [0, 0, 0]],
    [[0, 0, 1], [0, 0, 0], [1, 0, 0]],
    [[0, 0, -1j], [0, 0, 0], [1j, 0, 0]],
    [[0, 0, 0], [0, 0, 1], [0, 1, 0]],
    [[0, 0, 0], [0, 0, -1j], [0, 1j, 0]],
    [[1 / np.sqrt(3), 0, 0], [0, 1 / np.sqrt(3), 0], [0, 0, -2 / np.sqrt(3)]]
]

T = np.array(gm) / 2.0

def init_charge(q0):
    """
        Step 1: specific random color vector
    """
    Q0 = np.einsum('ijk,i', T, q0)

    """
        Step 2: create a random SU(3) matrix to rotate Q.
    """
    
    V = unitary_group.rvs(3)
    detV = np.linalg.det(V)
    U = V / detV ** (1 / 3)
    Ud = np.conj(U).T

    Q = np.einsum('ab,bc,cd', U, Q0, Ud)

    """
        Step 3: Project onto color components
    """

    q = 2 * np.einsum('ijk,kj', T, Q)
    return np.real(q)

- - -

In [3]:
# dependence = 'pT'
# pTs = [0, 1, 2, 5, 10]

dependence = 'Qs'
Qss = [1, 1.5, 2, 2.5, 3]

In [4]:
import numpy as np

# hbar * c [GeV * fm]
hbarc = 0.197326 

# Simulation box 
L = 10      
N = 512  
# N = 1024  
tau_sim = 1.5
DTS = 8
# DTS = 16     

# Glasma fields
su_group = 'su3'

Qs = 2        
ns = 50    
factor = 0.8        
g2mu = Qs / factor     
g = np.pi * np.sqrt(1 / np.log(Qs / 0.2))          		
mu = g2mu / g**2          	
ir = 0.1 * g**2 * mu  
 
uv = 10.0           

# Wong
# quark = 'charm'    
# mass = 1.275 
quark = 'beauty'    
mass = 4.18           
tau_form = 1/(2*mass)*hbarc

tau_sim += tau_form

initialization = 'toy'         
pT = 2    
ntp = 10**5  
nevents = 1    
representation = 'fundamental'      
boundary = 'periodic'       

# Results folder
folder = 'corr_' + initialization + '_' + dependence + '_dep_' + quark + '_q2_' + str(q2)      

# Store relevant parameters in a dictionary
p = {
    'QS': Qs,
    'TAU_SIM': tau_sim,
    'QUARK': quark,
    'MASS': mass,           
    'TFORM': tau_form,   
    'NTP':  ntp,    
    'PT': pT,          
    'FOLDER': folder, 
    'DEPENDENCE': dependence,        
    }   

In [5]:
import os
os.environ["MY_NUMBA_TARGET"] = "cuda"
os.environ["PRECISION"] = "double"
os.environ["GAUGE_GROUP"] = su_group

# Import relevant modules
import sys
sys.path.append('..')

# Glasma modules
import curraun.core as core
import curraun.mv as mv
import curraun.initial as initial
initial.DEBUG = False

import curraun.su as su
from curraun.numba_target import use_cuda
if use_cuda:
    from numba import cuda

# Wong modules
from curraun import wong
wong.BOUNDARY = boundary
wong.CUB_MOM = False
from curraun.wong import init_mom_toy, init_pos

from curraun.particles_correlators import Angles

Using CUDA
Using SU(3)
Using double precision
Using double precision


In [6]:
import pickle
from tqdm import tqdm

# Simulation routine
def simulate(p, ev):
    if p["DEPENDENCE"]=='pT':
        pT = p['PT']
    if p["DEPENDENCE"]=='Qs':
        pT = p['PT']
        Qs = p['QS']
        tau_sim = p['TAU_SIM']*3/Qs
        g2mu = Qs / 0.8     
        g = np.pi * np.sqrt(1 / np.log(Qs / 0.2))          		
        mu = g2mu / g**2          	
        ir = 0.1 * g2mu 

    output = {}
    output["parameters"] = p.copy()

    # Derived parameters
    a = L / N
    E0 = N / L * hbarc
    DT = 1.0 / DTS
    formt = int(tau_form / a * DTS)
    maxt = int(tau_sim / a * DTS)

    # Initialize Glasma fields
    s = core.Simulation(N, DT, g)
    va = mv.wilson(s, mu=mu / E0, m=ir / E0, uv=uv / E0, num_sheets=ns)
    vb = mv.wilson(s, mu=mu / E0, m=ir / E0, uv=uv / E0, num_sheets=ns)
    initial.init(s, va, vb)

    # Initialize the Wong solver
    wong_solver_q = wong.WongSolver(s, ntp)
    wong_solver_aq = wong.WongSolver(s, ntp)
    x0s_q, p0s_q, q0s_q = np.zeros((ntp, 3)), np.zeros((ntp, 5)), np.zeros((ntp, su.ALGEBRA_ELEMENTS))
    x0s_aq, p0s_aq, q0s_aq = np.zeros((ntp, 3)), np.zeros((ntp, 5)), np.zeros((ntp, su.ALGEBRA_ELEMENTS))
    masses = mass / E0 * np.ones(ntp)

    for i in range(ntp):
        if initialization=='toy':
            p0_q = init_mom_toy('pT', pT / E0)            

        p0_aq = [-p for p in p0_q]
        p0s_q[i, :], p0s_aq[i, :] = np.array(p0_q, dtype=object), np.array(p0_aq, dtype=object)

        x0_q, q0_q = init_pos(s.n), init_charge(q0)
        x0s_q[i, :], q0s_q[i, :] = x0_q, q0_q
        x0_aq, q0_aq = x0_q, init_charge(q0)
        x0s_aq[i, :], q0s_aq[i, :] = x0_aq, q0_aq

    wong_solver_q.initialize(x0s_q, p0s_q, q0s_q, masses)
    wong_solver_aq.initialize(x0s_aq, p0s_aq, q0s_aq, masses)

    qqbar_angle = Angles(wong_solver_q, wong_solver_aq, ntp)

    deta, dphi = np.zeros((maxt-formt, ntp)), np.zeros((maxt-formt, ntp))
    pTs = np.zeros((maxt-formt, ntp, 2))

    with tqdm(total=maxt) as pbar:
        for t in range(maxt):
            # Evolve Glasma fields
            core.evolve_leapfrog(s)

            # Solve Wong's equations
            if t>=formt:  
                qqbar_angle.compute()
                deta[t-formt] = qqbar_angle.deta.copy()
                dphi[t-formt] = qqbar_angle.dphi.copy()
                pTs[t-formt] = qqbar_angle.pT.copy() * E0
                
                wong_solver_q.evolve()
                wong_solver_aq.evolve()

            pbar.set_description("Event " + str(ev+1))
            pbar.update(1)

    tau = np.linspace(0, tau_sim-p['TFORM'], maxt-formt)
    output['tau'] = tau
    output['deta'], output['dphi'], output['pTs'] = deta, dphi, pTs

    return output

In [7]:
for ev in range(nevents):
    output = {}
    output["q2"], output["q3"] = q2, q3

    if dependence=='pT':
        output["pTs"] = pTs
        for (pTindex, pTvalue) in enumerate(pTs):
            p["PT"] = pTs[pTindex]
            print('pT=', pTvalue)
        
            output[pTvalue] = simulate(p, ev)

    if dependence=='Qs':
        output["Qss"] = Qss
        for (Qsindex, Qsvalue) in enumerate(Qss):
            p["QS"] = Qss[Qsindex]
            print('Qs=', Qsvalue)
        
            output[Qsvalue] = simulate(p, ev)

    current_path = os.getcwd() 
    results_folder = 'results'
    if not os.path.isdir(results_folder):
        os.makedirs(results_folder)
    results_path = current_path + '/' + results_folder
    os.chdir(results_path)

    wong_folder = p['FOLDER']
    if not os.path.isdir(wong_folder):
        os.makedirs(wong_folder)
    wong_path = results_path + '/' + wong_folder + '/'
    os.chdir(wong_path)

    filename = 'event_' + str(ev+1) + '.pickle'
    with open(filename, 'wb') as handle:
        pickle.dump(output, handle)  

Qs= 1


Event 1: 100%|██████████| 1872/1872 [16:28<00:00,  1.89it/s]


Qs= 1.5


Event 1: 100%|██████████| 1248/1248 [09:42<00:00,  2.14it/s]


Qs= 2


Event 1: 100%|██████████| 936/936 [07:11<00:00,  2.17it/s]


Qs= 2.5


Event 1: 100%|██████████| 748/748 [05:52<00:00,  2.12it/s]


Qs= 3


Event 1: 100%|██████████| 624/624 [04:45<00:00,  2.18it/s]
